In [11]:
import re
import pandas as pd

# Input JSON table
table_data = [
    ["", "", "Years Ended September 30,", ""],
    ["", "2019", "2018", "2017"],
    ["Fixed Price", "$  1,452.4", "$  1,146.2", "$  1,036.9"],
    ["Other", "44.1", "56.7", "70.8"],
    ["Total sales", "$1,496.5", "$1,202.9", "$1,107.7"]
]

def parse_value(value):
    """Parse value to separate numeric value and unit."""
    match = re.match(r'([^\d.-]*)([\d,.]+)', value.strip())
    if match:
        unit = match.group(1).strip()
        num_value = float(match.group(2).replace(',', ''))
        return num_value, unit
    return None, value

def convert_table(table):
    headers = table[0][2], table[1][1:]  # Extract column labels
    result = []

    for row in table[2:]:
        row_label = row[0]
        for i, col_value in enumerate(row[1:]):
            value, unit = parse_value(col_value)
            result.append({
                "value": value,
                "unit": unit,                
                "Column_Labels": [headers[0], headers[1][i]],
                "Row_Labels": [row_label]
            })

    return {"table": result}

# Convert the table
converted_table = convert_table(table_data)

pd.DataFrame(converted_table['table'])
# Output the converted table
#import json
#print(json.dumps(converted_table, indent=4))


,value,unit,Column_Labels,Row_Labels
0,1452.4,$,"[Years Ended September 30,, 2019]",[Fixed Price]
1,1146.2,$,"[Years Ended September 30,, 2018]",[Fixed Price]
2,1036.9,$,"[Years Ended September 30,, 2017]",[Fixed Price]
3,44.1,,"[Years Ended September 30,, 2019]",[Other]
4,56.7,,"[Years Ended September 30,, 2018]",[Other]
5,70.8,,"[Years Ended September 30,, 2017]",[Other]
6,1496.5,$,"[Years Ended September 30,, 2019]",[Total sales]
7,1202.9,$,"[Years Ended September 30,, 2018]",[Total sales]
8,1107.7,$,"[Years Ended September 30,, 2017]",[Total sales]


In [17]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [12]:
# Data representation
data = [
    {"value": 1452.4, "unit": "$", "Row_Labels": ["Fixed Price"], "Column_Labels": ["Years Ended September 30,", "2019"]},
    {"value": 1146.2, "unit": "$", "Row_Labels": ["Fixed Price"], "Column_Labels": ["Years Ended September 30,", "2018"]},
    {"value": 1452.4, "unit": "$", "Row_Labels": ["Fixed Price"], "Column_Labels": ["Years Ended September 30,", "2017"]},
    {"value": 44.1, "unit": "", "Row_Labels": ["Other"], "Column_Labels": ["Years Ended September 30,", "2019"]},
    {"value": 56.7, "unit": "", "Row_Labels": ["Other"], "Column_Labels": ["Years Ended September 30,", "2018"]},
    {"value": 70.8, "unit": "", "Row_Labels": ["Other"], "Column_Labels": ["Years Ended September 30,", "2017"]},
    {"value": 1496.5, "unit": "$", "Row_Labels": ["Total sales"], "Column_Labels": ["Years Ended September 30,", "2019"]},
    {"value": 1202.9, "unit": "$", "Row_Labels": ["Total sales"], "Column_Labels": ["Years Ended September 30,", "2018"]},
    {"value": 1107.7, "unit": "$", "Row_Labels": ["Total sales"], "Column_Labels": ["Years Ended September 30,", "2017"]}
]

# Extracting revenue for "Fixed Price" in 2018
linkedin_revenue_2018 = None
for entry in data:
    if entry["Row_Labels"] == ["Fixed Price"] and entry["Column_Labels"] == ["Years Ended September 30,", "2018"]:
        linkedin_revenue_2018 = entry["value"]
        break

# Output result
print(f"Revenue from LinkedIn in 2018: ${linkedin_revenue_2018} million")


Revenue from LinkedIn in 2018: $1146.2 million


In [13]:
def extract_table_data(table):
    # Identify the row that contains the years
    years_row = None
    for i, row in enumerate(table):
        if all(cell.isdigit() or cell.strip() == '' for cell in row[1:]):  # Detect year row
            years_row = table[i]
            break
    
    # Extract years from the header row (ignoring the first two empty cells)
    years = years_row[1:]

    extracted_data = []
    current_section = ""

    # Iterate through the rows starting from the first data row
    for row in table[3:]:
        if all(cell.strip() == '' for cell in row):  # Skip empty rows
            continue

        # Detect section headers, like "Transportation Solutions"
        if row[0].endswith(":"):
            current_section = row[0].strip(":")
            continue

        category = row[0].strip()  # The first cell is the category
        values = row[1:]           # The rest of the row contains the values

        # Combine section, category, year, and value
        for year, value in zip(years, values):
            if value.strip():  # Only add non-empty values
                extracted_data.append({
                    "Section": current_section,
                    "Category": category,
                    "Year": year,
                    "Value": value.strip()
                })
    
    return extracted_data

# Example table input
table = [
    ['', '', 'Fiscal', ''],
    ['', '2019', '2018', '2017'],
    ['', '', '(in millions)', ''],
    ['Transportation Solutions:', '', '', ''],
    ['Automotive', '$ 5,686', '$ 6,092', '$  5,228'],
    ['Commercial transportation', '1,221', '1,280', '997'],
    ['Sensors', '914', '918', '814'],
    ['Total Transportation Solutions', '7,821', '8,290', '7,039'],
    ['Industrial Solutions:', '', '', ''],
    ['Industrial equipment', '1,949', '1,987', '1,747'],
    ['Aerospace, defense, oil, and gas', '1,306', '1,157', '1,075'],
    ['Energy', '699', '712', '685'],
    ['Total Industrial Solutions', '3,954', '3,856', '3,507'],
    ['Communications Solutions:', '', '', ''],
    ['Data and devices', '993', '1,068', '963'],
    ['Appliances', '680', '774', '676'],
    ['Total Communications Solutions', '1,673', '1,842', '1,639'],
    ['Total', '$ 13,448', '$ 13,988', '$ 12,185']
]

# Call the function and print the results
data = extract_table_data(table)
for item in data:
    print(f"Section: {item['Section']}, Category: {item['Category']}, Year: {item['Year']}, Value: {item['Value']}")


Section: Transportation Solutions, Category: Automotive, Year: 2019, Value: $ 5,686
Section: Transportation Solutions, Category: Automotive, Year: 2018, Value: $ 6,092
Section: Transportation Solutions, Category: Automotive, Year: 2017, Value: $  5,228
Section: Transportation Solutions, Category: Commercial transportation, Year: 2019, Value: 1,221
Section: Transportation Solutions, Category: Commercial transportation, Year: 2018, Value: 1,280
Section: Transportation Solutions, Category: Commercial transportation, Year: 2017, Value: 997
Section: Transportation Solutions, Category: Sensors, Year: 2019, Value: 914
Section: Transportation Solutions, Category: Sensors, Year: 2018, Value: 918
Section: Transportation Solutions, Category: Sensors, Year: 2017, Value: 814
Section: Transportation Solutions, Category: Total Transportation Solutions, Year: 2019, Value: 7,821
Section: Transportation Solutions, Category: Total Transportation Solutions, Year: 2018, Value: 8,290
Section: Transportation

In [14]:
extract_table_data(table_data)

[{'Section': '', 'Category': 'Other', 'Year': '2019', 'Value': '44.1'},
 {'Section': '', 'Category': 'Other', 'Year': '2018', 'Value': '56.7'},
 {'Section': '', 'Category': 'Other', 'Year': '2017', 'Value': '70.8'},
 {'Section': '',
  'Category': 'Total sales',
  'Year': '2019',
  'Value': '$1,496.5'},
 {'Section': '',
  'Category': 'Total sales',
  'Year': '2018',
  'Value': '$1,202.9'},
 {'Section': '',
  'Category': 'Total sales',
  'Year': '2017',
  'Value': '$1,107.7'}]

In [15]:
extract_table_data([['', 'Domestic', '', 'International', ''],
 ['', 'September 30,', '', 'September 30,', ''],
 ['', '2019', '2018', '2019', '2018'],
 ['Discount rate', '4.00%', '3.75%', '1.90%', '2.80%'],
 ['Expected return on plan assets', '', '', '3.40%', '3.70%'],
 ['Rate of compensation increase', '', '', '- - %', '- - %']])

[{'Section': '',
  'Category': 'Discount rate',
  'Year': '2019',
  'Value': '4.00%'},
 {'Section': '',
  'Category': 'Discount rate',
  'Year': '2018',
  'Value': '3.75%'},
 {'Section': '',
  'Category': 'Discount rate',
  'Year': '2019',
  'Value': '1.90%'},
 {'Section': '',
  'Category': 'Discount rate',
  'Year': '2018',
  'Value': '2.80%'},
 {'Section': '',
  'Category': 'Expected return on plan assets',
  'Year': '2019',
  'Value': '3.40%'},
 {'Section': '',
  'Category': 'Expected return on plan assets',
  'Year': '2018',
  'Value': '3.70%'},
 {'Section': '',
  'Category': 'Rate of compensation increase',
  'Year': '2019',
  'Value': '- - %'},
 {'Section': '',
  'Category': 'Rate of compensation increase',
  'Year': '2018',
  'Value': '- - %'}]

In [16]:
def extract_table_data(table):
    # Find the row where the years are located
    years = []
    header_rows = []
    
    # Identify the header rows by detecting the first numeric year in each row
    for i, row in enumerate(table):
        if any(cell.strip().isdigit() for cell in row):
            header_rows.append(i)

    if len(header_rows) < 2:
        print("Invalid table format")
        return []
    
    # Extract metadata from the first few rows
    # In the second table, the first row defines Domestic and International headers
    categories = table[0][1:]  # Skip first cell
    years_dom = table[header_rows[1]][1:]  # Take years under "Domestic"
    
    extracted_data = []
    
    # Iterate through the data rows (below the headers)
    for row in table[header_rows[-1] + 1:]:
        category = row[0].strip()  # First cell is the category
        values = row[1:]           # The rest are values
        
        # Combine category and value, adding appropriate year and domestic/international metadata
        for cat, year, value in zip(categories, years_dom, values):
            if value.strip():  # Only add non-empty values
                extracted_data.append({
                    "Category": category,
                    "Region/Type": cat,
                    "Year": year,
                    "Value": value.strip()
                })
    
    return extracted_data

In [18]:
for i in range(10):
    print(extract_table_data(devdf.loc(0)[i]['table']['table']))
    

Invalid table format
[]
[{'Category': 'Total Communications Solutions', 'Region/Type': '', 'Year': '1,221', 'Value': '1,673'}, {'Category': 'Total Communications Solutions', 'Region/Type': 'Fiscal', 'Year': '1,280', 'Value': '1,842'}, {'Category': 'Total Communications Solutions', 'Region/Type': '', 'Year': '997', 'Value': '1,639'}, {'Category': 'Total', 'Region/Type': '', 'Year': '1,221', 'Value': '$ 13,448'}, {'Category': 'Total', 'Region/Type': 'Fiscal', 'Year': '1,280', 'Value': '$ 13,988'}, {'Category': 'Total', 'Region/Type': '', 'Year': '997', 'Value': '$ 12,185'}]
Invalid table format
[]
[]
[]
[{'Category': 'Total liabilities', 'Region/Type': '2019', 'Year': '128', 'Value': '$909'}, {'Category': 'Total liabilities', 'Region/Type': '2018', 'Year': '126', 'Value': '$1,305'}]
[{'Category': 'Other', 'Region/Type': 'December 31,', 'Year': '592', 'Value': '6,723'}, {'Category': 'Other', 'Region/Type': '', 'Year': '519', 'Value': '5,711'}, {'Category': '', 'Region/Type': 'December 31,